In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [8]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
tf.logging.set_verbosity(tf.logging.ERROR)
np.random.seed(6)

In [4]:
db = pd.read_csv('../social_listening1/original_tweets.csv', lineterminator='\n')
db.head()

,_id,created_at,favorite_count,full_text,retweet_count,u4u_dataset,user.created_at,user.description,user.followers_count,user.friends_count,user.lang,user.listed_count,user.location,user.name,user.screen_name
0,5bd86c7968a761d62501fda2,Sun Sep 23 15:01:52 +0000 2018,1,2016\nRemember when #PeterDutton was found gui...,0,asylumseeker,Wed Oct 02 00:18:21 +0000 2013,This page is sharing information about the Lib...,6490,5435,en,261,Western Australia,TALAOLP,Talaolp
1,5bd86c7968a761d62501fda5,Sun Sep 23 11:09:04 +0000 2018,1,"This pains me, but it's time to compromise on ...",1,asylumseeker,Sat Nov 28 02:34:51 +0000 2009,"big fan of irreverent political commentary, c...",1966,3281,en,125,"country Victoria, Australia",eithne,eithne52
2,5bd86c7968a761d62501fda9,Sun Sep 23 02:01:55 +0000 2018,7,What about all his other well paid jobs! He ha...,2,asylumseeker,Mon Oct 22 07:25:28 +0000 2012,#Wiimpitja - black fella #BarkindjiNation #Kal...,8583,5514,en,358,NaN,Paul Dutton,pauldutton1968
3,5bd86c7968a761d62501fdab,Sun Sep 23 00:47:55 +0000 2018,2,A must read analysis of policy paralysis on as...,2,asylumseeker,Sat Aug 03 03:08:39 +0000 2013,"Senior Counsel, AWL Woman Lawyer of the Year, ...",5019,932,en,85,Australia,Fi McLeod SC,FiMcLeodSC
4,5bd86c7968a761d62501fdaf,Sat Sep 22 02:49:45 +0000 2018,0,"""@halyapuff: #Ukrainian prosecutor general adm...",0,asylumseeker,Fri Feb 04 22:40:35 +0000 2011,NaN,2097,1853,en,83,THE MOON,Gaby Skittles friend,GABchaag10


In [5]:
text = db['full_text']

In [9]:
samples = np.random.choice(np.arange(1, text.shape[0]), 50000, replace=False)

In [10]:
documents = text.take(samples).values

In [11]:
for i in range(len(documents)):
    documents[i] = documents[i].replace('\n', ' ') 
    documents[i] = documents[i].replace('\r', ' ') 

## Universal Sentence Encoder embeddings

In [29]:
embed = hub.Module(module_url)
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    embeddings = session.run(embed(documents))

In [31]:
np.savetxt("embeds.tsv", embeddings, delimiter="\t")

In [59]:
np.savetxt("tweets.tsv", documents, fmt="%s")

In [60]:
documents.shape

(50000,)

In [61]:
i = 0
with open('tweets.tsv', 'r') as f:
    for l in f.readlines():
        i += 1
i

50000

## Sentiment Analysis

In [15]:
import nltk 
import pandas as pd 
import string

from nltk.sentiment.vader import SentimentIntensityAnalyzer

messages = documents
sid = SentimentIntensityAnalyzer()

summary = {"positive":0,"neutral":0,"negative":0}
for x in messages: 
    ss = sid.polarity_scores(x)
    if ss["compound"] == 0.0: 
        summary["neutral"] +=1
    elif ss["compound"] > 0.0:
        summary["positive"] +=1
    else:
        summary["negative"] +=1
print(summary)

{'positive': 17698, 'neutral': 8923, 'negative': 23379}


In [13]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...


True